# Load the daily updated dataset

In [46]:
# Azure storage access info
blob_account_name = "pandemicdatalake"
blob_container_name = "public"
blob_relative_path = "curated/covid-19/ecdc_cases/latest/ecdc_cases.parquet"
blob_sas_token = r""

# Allow SPARK to read from Blob remotely
wasbs_path = 'wasbs://%s@%s.blob.core.windows.net/%s' % (blob_container_name, blob_account_name, blob_relative_path)
spark.conf.set(
  'fs.azure.sas.%s.%s.blob.core.windows.net' % (blob_container_name, blob_account_name),
  blob_sas_token)
print('Remote blob path: ' + wasbs_path)

# SPARK read parquet, note that it won't load any data yet by now
df = spark.read.parquet(wasbs_path)
print('Register the DataFrame as a SQL temporary view: source')
df.createOrReplaceTempView('source')

# Display top 10 rows
print('Displaying top 10 rows: ')
display(spark.sql('SELECT * FROM source LIMIT 10'))

StatementMeta(smallPool, 12, 46, Finished, Available)

Remote blob path: wasbs://public@pandemicdatalake.blob.core.windows.net/curated/covid-19/ecdc_cases/latest/ecdc_cases.parquet
Register the DataFrame as a SQL temporary view: source
Displaying top 10 rows: 


SynapseWidget(Synapse.DataFrame, c3d45b38-6ec6-448e-a681-618061a25136)

In [47]:
spark.sql('SELECT * FROM source order by date_rep desc LIMIT 10').show(vertical=True)

StatementMeta(smallPool, 12, 47, Finished, Available)

-RECORD 0-----------------------------------------
 date_rep                  | 2020-12-14           
 day                       | 14                   
 month                     | 12                   
 year                      | 2020                 
 cases                     | 0                    
 deaths                    | 0                    
 countries_and_territories | Anguilla             
 geo_id                    | AI                   
 country_territory_code    | AIA                  
 pop_data_2018             | null                 
 continent_exp             | America              
 load_date                 | 2021-07-06 00:07:... 
 iso_country               | AI                   
-RECORD 1-----------------------------------------
 date_rep                  | 2020-12-14           
 day                       | 14                   
 month                     | 12                   
 year                      | 2020                 
 cases                     | 17

In [48]:
df.count()

StatementMeta(smallPool, 12, 48, Finished, Available)

61900

61900 records in total in our updated dataset and the load date is 6th of July in this case

# Existing Datalake (dataset uploaded on the 22nd of June 2021)

Setup the path location where the parquet file is stored

In [51]:
storagePath = "abfss://churning@churninge2edemo.dfs.core.windows.net/synapse/tables/covid"

StatementMeta(smallPool, 12, 51, Finished, Available)

In [52]:
dfcovidParquet = spark.read.load(f"{storagePath}/ecdc_cases.parquet", format="parquet")

StatementMeta(smallPool, 12, 52, Finished, Available)

# Convert the parquet file to delta

In [53]:
dfcovidParquet.write.format("delta").mode("overwrite").save(f"{storagePath}/delta")

StatementMeta(smallPool, 12, 53, Finished, Available)

In [54]:
dfCovidDelta =  spark.read.load(f"{storagePath}/delta", format="delta")

StatementMeta(smallPool, 12, 54, Finished, Available)

In [55]:
dfCovidDelta.show(vertical=True)

StatementMeta(smallPool, 12, 55, Finished, Available)

-RECORD 0-----------------------------------------
 date_rep                  | 2020-12-14           
 day                       | 14                   
 month                     | 12                   
 year                      | 2020                 
 cases                     | 746                  
 deaths                    | 6                    
 countries_and_territories | Afghanistan          
 geo_id                    | AF                   
 country_territory_code    | AFG                  
 pop_data_2018             | null                 
 continent_exp             | Asia                 
 load_date                 | 2021-06-22 00:05:... 
 iso_country               | AF                   
-RECORD 1-----------------------------------------
 date_rep                  | 2020-12-13           
 day                       | 13                   
 month                     | 12                   
 year                      | 2020                 
 cases                     | 29

In [56]:
dfCovidDelta.count()

StatementMeta(smallPool, 12, 56, Finished, Available)

61900

61900 records in total in our datalake

We have now 2 Datasets:
<br> One from the 22nd of June: **dfCovidDelta**
<br> One based on the daily updated: **df**

# Check any difference between these two datasets in term of cases and deaths for the same country and date

In [57]:
dfDiff = dfCovidDelta.join(df,(dfCovidDelta.date_rep ==  df.date_rep) & (dfCovidDelta.iso_country == df.iso_country) & ((dfCovidDelta.cases !=  df.cases) | (dfCovidDelta.deaths !=  df.deaths))).select("*")

StatementMeta(smallPool, 12, 57, Finished, Available)

In [58]:
dfDiff.show(vertical=True)

StatementMeta(smallPool, 12, 58, Finished, Available)

(0 rows)

There is no differences in term of cases, detahs for the same country
<br> The dataset contains only data until 14/12/2020. See https://www.ecdc.europa.eu/en/covid-19/data-collection
<br> The only difference is the load date

# Create a Delta table from DataLake location

In [59]:
from delta.tables import *

deltaCovidTable = DeltaTable.forPath(spark,f"{storagePath}/delta")

StatementMeta(smallPool, 12, 59, Finished, Available)

In [60]:
from pyspark.sql.functions import to_date, col

StatementMeta(smallPool, 12, 60, Finished, Available)

In [61]:
deltaCovidTable.toDF().withColumn("load_date_dateFormat",to_date(col("load_date"),"yyyy-MM-dd")).show(vertical=True)

StatementMeta(smallPool, 12, 61, Finished, Available)

-RECORD 0-----------------------------------------
 date_rep                  | 2020-12-14           
 day                       | 14                   
 month                     | 12                   
 year                      | 2020                 
 cases                     | 746                  
 deaths                    | 6                    
 countries_and_territories | Afghanistan          
 geo_id                    | AF                   
 country_territory_code    | AFG                  
 pop_data_2018             | null                 
 continent_exp             | Asia                 
 load_date                 | 2021-06-22 00:05:... 
 iso_country               | AF                   
 load_date_dateFormat      | 2021-06-22           
-RECORD 1-----------------------------------------
 date_rep                  | 2020-12-13           
 day                       | 13                   
 month                     | 12                   
 year                      | 20

# Merge the two datasets to keep in our datalake only the the most updated records

In [62]:

deltaCovidTable.alias("deltaLake").merge(df.alias("dataUpdate"),condition="deltaLake.date_rep = dataUpdate.date_rep and deltaLake.iso_country = dataUpdate.iso_country").whenMatchedUpdate(set = {"cases":"dataUpdate.cases","load_date": "dataUpdate.load_date"}).execute()


StatementMeta(smallPool, 12, 62, Finished, Available)

# Check if after the merge we have still data loaded on the 22nd of June

In [63]:
deltaCovidTable.toDF().withColumn("load_date_dateFormat",to_date(col("load_date"),"yyyy-MM-dd")).where("load_date_dateFormat = '2021-06-22'").show(vertical=True)

StatementMeta(smallPool, 12, 63, Finished, Available)

(0 rows)

# Check if after the merge we have data from the last updates

In [64]:
deltaCovidTable.toDF().withColumn("load_date_dateFormat",to_date(col("load_date"),"yyyy-MM-dd")).where("load_date_dateFormat = '2021-07-06'").show(vertical=True)

StatementMeta(smallPool, 12, 64, Finished, Available)

-RECORD 0-----------------------------------------
 date_rep                  | 2020-12-14           
 day                       | 14                   
 month                     | 12                   
 year                      | 2020                 
 cases                     | 746                  
 deaths                    | 6                    
 countries_and_territories | Afghanistan          
 geo_id                    | AF                   
 country_territory_code    | AFG                  
 pop_data_2018             | null                 
 continent_exp             | Asia                 
 load_date                 | 2021-07-06 00:07:... 
 iso_country               | AF                   
 load_date_dateFormat      | 2021-07-06           
-RECORD 1-----------------------------------------
 date_rep                  | 2020-12-13           
 day                       | 13                   
 month                     | 12                   
 year                      | 20

In [65]:
deltaCovidTable.toDF().count()

StatementMeta(smallPool, 12, 65, Finished, Available)

61900

After the merge our dataset still contain 61900 records...but only from the most updated dataset